# Training Abstract2Title

In [1]:
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
# import wandb
from datasets import load_from_disk, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, \
                         DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/liana/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# # replace with your weights and biases username otherwise comment this
# wandb.init(project="abstract-to-title", entity="nerdimite")

## Preprocess Data

In [4]:
# Initialize T5-base tokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [5]:
# Load the processed data
dataset = load_from_disk('arxiv_AI_dataset')

In [6]:
MAX_SOURCE_LEN = 512
MAX_TARGET_LEN = 128

In [7]:
def preprocess_data(example):
    
    model_inputs = tokenizer(example['abstract'], max_length=MAX_SOURCE_LEN, padding=True, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['title'], max_length=MAX_TARGET_LEN, padding=True, truncation=True)

    # Replace all pad token ids in the labels by -100 to ignore padding in the loss
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs['labels'] = labels["input_ids"]

    return model_inputs

In [8]:
# Apply preprocess_data() to the whole dataset
processed_dataset = dataset.map(
    preprocess_data,
    batched=True,
    remove_columns=['abstract', 'title'],
    desc="Running tokenizer on dataset",
)

processed_dataset

Running tokenizer on dataset:   0%|          | 0/37 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 36074
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 2005
    })
    val: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 2004
    })
})

## Training Parameters

In [9]:
batch_size = 8
num_epochs = 5
learning_rate = 5.6e-5
weight_decay = 0.01
log_every = 50
eval_every = 1000
lr_scheduler_type = "linear"

In [10]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="model-t5-base",
    evaluation_strategy="steps",
    eval_steps=eval_every,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_steps=500,
    save_total_limit=3,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    logging_steps=log_every,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb",
    resume_from_checkpoint=True,
)

/home/liana/projects/abstract-to-title-generator/venv/lib/python3.9/site-packages/torch/cuda/__init__.py:143: UserWarning: 
NVIDIA GeForce RTX 3050 Ti Laptop GPU with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA GeForce RTX 3050 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


## Train

In [11]:
# Initialize T5-base model
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [12]:
# Define ROGUE metrics on evaluation data
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Compute ROUGE scores and get the median scores
    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    return {k: round(v, 4) for k, v in result.items()}

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [13]:
# Dynamic padding in batch using a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [14]:
# Define the trainer
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
%%time
trainer.train()

***** Running training *****
  Num examples = 36074
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 22550
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lianaling (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## Evaluate

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('model-t5-base/checkpoint-6000/').to('cuda')
tokenizer = AutoTokenizer.from_pretrained('model-t5-base/checkpoint-6000/')

In [ ]:
%%time
trainer.evaluate(eval_dataset=processed_dataset['test'])

***** Running Evaluation *****
  Num examples = 2005
  Batch size = 8


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


CPU times: user 3min 40s, sys: 194 ms, total: 3min 40s
Wall time: 3min 40s


{'eval_loss': 1.6707532405853271,
 'eval_rouge1': 47.1746,
 'eval_rouge2': 26.8231,
 'eval_rougeL': 41.7727,
 'eval_rougeLsum': 41.8263,
 'eval_runtime': 220.717,
 'eval_samples_per_second': 9.084,
 'eval_steps_per_second': 1.137}

## Predict

In [ ]:
temperature = 0.9
num_beams = 4
max_gen_length = 128

In [ ]:
abstract = """In this paper, we question if self-supervised learning provides
new properties to Vision Transformer (ViT) [19] that
stand out compared to convolutional networks (convnets).
Beyond the fact that adapting self-supervised methods to this
architecture works particularly well, we make the following
observations: first, self-supervised ViT features contain
explicit information about the semantic segmentation of an
image, which does not emerge as clearly with supervised
ViTs, nor with convnets. Second, these features are also excellent
k-NN classifiers, reaching 78.3% top-1 on ImageNet
with a small ViT. Our study also underlines the importance of
momentum encoder [33], multi-crop training [10], and the
use of small patches with ViTs. We implement our findings
into a simple self-supervised method, called DINO, which
we interpret as a form of self-distillation with no labels.
We show the synergy between DINO and ViTs by achieving
80.1% top-1 on ImageNet in linear evaluation with ViT-Base"""
# abstract = dataset['test'][0]['abstract']
inputs = tokenizer([abstract], max_length=512, return_tensors='pt')

title_ids = model.generate(
    inputs['input_ids'].to('cuda'), 
    num_beams=num_beams, 
    temperature=temperature, 
    max_length=max_gen_length, 
    early_stopping=True
)
title = tokenizer.decode(title_ids[0].tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(title)

Self-Supervised Learning of Vision Transformers
